In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
import numpy as np

# 1. Parameters
dataset_path = 'Rice_Image_Dataset' 
batch_size = 32
img_size = (64, 64)

# 2. Load dataset (train and validation split)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)


# 3. find number of class
num_classes = len(train_dataset.class_names)

# 4. Normalize pixel values (0-255 to 0-1)
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


#-----------------------------------------
# tf.keras.layers.Rescaling(1./255) creates a layer that divides every pixel value by 255.

# Since pixel values range from 0 to 255, dividing by 255 changes the range to 0 to 1.

# .map() is a method to apply a function to every element in the dataset.

# Here, each element is a tuple (images, labels) where:

# images is a batch of images.

# labels is the corresponding batch of labels.

# lambda x, y: (normalization_layer(x), y) is a short function that:

# Takes images x and labels y.

# Applies the rescaling (divides all pixels in x by 255).

# Returns the normalized images along with the original labels unchanged.
#-----------------------------------------

# 5. Flatten images for ANN
#Dense layers in an ANN expect inputs as 1D vectors, not 3D arrays.
#Flattening converts image data into a suitable format for these layers.

def flatten_images(images, labels):
    # Flatten each image from (batch_size, 64, 64, 3) to (batch_size, 64*64*3)
    images = tf.reshape(images, [tf.shape(images)[0], -1])
    return images, labels


train_dataset = train_dataset.map(flatten_images)
val_dataset = val_dataset.map(flatten_images)

# 6. Build ANN model
model = Sequential([
    Input(shape=(img_size[0] * img_size[1] * 3,)),  # flattened input
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # labels are integers
              metrics=['accuracy'])

# 7. Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=20)


Found 75000 files belonging to 5 classes.
Using 60000 files for training.
Found 75000 files belonging to 5 classes.
Using 15000 files for validation.
Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.9520 - loss: 0.1408 - val_accuracy: 0.9621 - val_loss: 0.1176
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9723 - loss: 0.0812 - val_accuracy: 0.9668 - val_loss: 0.0973
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9775 - loss: 0.0659 - val_accuracy: 0.9647 - val_loss: 0.1037
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9819 - loss: 0.0539 - val_accuracy: 0.9811 - val_loss: 0.0583
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9835 - loss: 0.0481 - val_accuracy: 0.9872 - val_loss: 0.0402
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9853 - loss: 0.0431 - val_accuracy: 0.9866 - val_loss: 0.0426
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accurac

In [5]:
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy:.4f}')


469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9863 - loss: 0.0476
Validation Loss: 0.0476
Validation Accuracy: 0.9863
